
#### 1.1 How many goals were attempted by head
#### 1.2 How many matches had attendees more than 12000
#### 2.1 Shots list where goal was scored outside 16m box
#### 2.2 Rank teams based on average attendance at home games 
#### 2.2* 
calc goals for each team, for home and away correspondingly
which teams were most advantaged when playing at home
average difference of goals per team [ex. 3-1(2) 4-7(-3) 2-3/2 = -0.5 ]
persentage of home/away goals

In [1]:
from MongoStruct import *  

### Load all preprocessed data, dataframes and collections

In [2]:
client,shots_collection,matches_collection,players_collection,teams_collection,df_shots,df_matches,df_players,df_teams=get_all_data()


Database dropped
shots Collection dropped
matches Collection dropped
players Collection dropped
teams Collection dropped
# shots documents: 245180
# matches documents: 14148
# players documents: 20414
# teams documents: 675


In [3]:
# print(df_shots.columns)
# print(df_shots.head(5))
# print(df_matches.columns)
# print(df_matches.head(5))
# print(df_players.columns)
# print(df_players.head(-5))
# print(df_teams.columns)
# print(df_teams.head(5))

# 1. How many goals were attempted by head

In [4]:
# Define the query
myquery = {"Outcome": "Goal" ,'Body Part':'Head'}

# Execute the query
result = shots_collection.find(myquery)

print(result[0])

# # Print the results
# for i in range(5):
#     print(result[i])

# print(results.alive)
# print(type(results))
cursor_list = list(result)
print(f"Number of documents in cursor: {len(cursor_list)}")

{'_id': ObjectId('6747a0e40da0248efe0d9387'), 'Player': '4a1a9578', 'Outcome': 'Goal', 'Distance': 8.0, 'Body Part': 'Head', 'match_id': 'de515487'}
Number of documents in cursor: 3963


# 2. How many matches had attendees more than 12000

In [5]:
# Define the query
myquery = {"Attendance":{"$gt": 12000}}

# Execute the query
result = matches_collection.find(myquery)

print(result[0])

# # Print the results
# for i in range(5):
#     print(result[i])

# print(results.alive)
# print(type(results))
cursor_list = list(result)
print(f"Number of documents in cursor: {len(cursor_list)}")

{'_id': ObjectId('6747a0e60da0248efe115144'), 'id': '0006415c', 'Date': '2022-10-09', 'league': 11, 'Round': 11, 'Day': 'Sun', 'Attendance': 24671, 'home_id': '04eea015', 'away_id': '922493f3', 'score_away': 2, 'score_home': 2, 'position_home': 59, 'position_away': 41, 'home_capitan': '/en/players/75f2c59f/Roberto-Pereyra', 'away_capitan': '/en/players/2a1beb34/Marten-de-Roon', 'home_xg': 1.0, 'away_xg': 2.1, 'home_xga': 2.1, 'away_xga': 1.0, 'home_formation': '3-5-2', 'away_formation': '3-4-1-2', 'Timestamp': '2022-10-09 15:00:00'}
Number of documents in cursor: 9166


In [6]:
pipeline = [
    {
        '$match': {
            'Outcome': 'Goal'  # Filter only for documents where Outcome is 'Goal'
        }
    },
    {
        '$group': {
            '_id': {
                'Player': '$Player',  # Group by Player
                'Outcome': '$Outcome'  # Include Outcome in the grouping (always 'Goal' )
            },
            'distance_gt_16': {
                '$sum': {
                    '$cond': [{'$gt': ['$Distance', 16]}, 1, 0]  # Count distances > 16
                }
            },
            'player_count': {'$sum': 1}  # Count number of players in each group
        }
    }
]

print(f"\nResults for GROUP query:")
result = list(shots_collection.aggregate(pipeline))
for doc in result:
    print(f"Player: {doc['_id']['Player']}, Distance > 16 Count: {doc['distance_gt_16']}, Player Count: {doc['player_count']}")




Results for GROUP query:
Player: c693347d, Distance > 16 Count: 1, Player Count: 15
Player: 102e2599, Distance > 16 Count: 3, Player Count: 6
Player: bfae110f, Distance > 16 Count: 0, Player Count: 5
Player: 9ef42479, Distance > 16 Count: 1, Player Count: 10
Player: 40ef7347, Distance > 16 Count: 0, Player Count: 4
Player: cdf65d12, Distance > 16 Count: 1, Player Count: 1
Player: 3626f7d9, Distance > 16 Count: 0, Player Count: 2
Player: 4e358a2d, Distance > 16 Count: 0, Player Count: 9
Player: 5515376c, Distance > 16 Count: 2, Player Count: 6
Player: b418dbd4, Distance > 16 Count: 5, Player Count: 33
Player: d2c12fe0, Distance > 16 Count: 0, Player Count: 1
Player: 99d660b2, Distance > 16 Count: 1, Player Count: 4
Player: 562a4799, Distance > 16 Count: 0, Player Count: 2
Player: 4207e578, Distance > 16 Count: 0, Player Count: 5
Player: dc3a9a0d, Distance > 16 Count: 1, Player Count: 2
Player: 85bed4ee, Distance > 16 Count: 0, Player Count: 2
Player: 5bd72174, Distance > 16 Count: 5, P